In [1]:
import pandas as pd
import pyrankvote
from pathlib import Path
from random import seed
seed(252)

In [2]:
df = pd.read_csv(Path('./data/20210109.csv').resolve())
df

,Timestamp,Rank your movie choices. You don't have to do a full ranking. [First Choice],Rank your movie choices. You don't have to do a full ranking. [Second Choice],Rank your movie choices. You don't have to do a full ranking. [Third Choice],Rank your movie choices. You don't have to do a full ranking. [Fourth Choice],Rank your movie choices. You don't have to do a full ranking. [Fifth Choice],What theme should we explore next week? [First Choice],What theme should we explore next week? [Second Choice],What theme should we explore next week? [Third Choice],What theme should we explore next week? [Fourth Choice],What theme should we explore next week? [Fifth Choice]
0,1/6/2021 9:28:28,Nomadland (2020),Raising Arizona (1987),This Must Be The Place (2011),Hidden Agenda (1990),Lone Star (1996),ANIMALS,[Default to Nicolas Cage],ALEC BALDWIN: THE SEXY YEARS,VAMPIRES! FUCK!,HORNINESS IS NOT A CRIME
1,1/6/2021 9:40:40,Lone Star (1996),Nomadland (2020),Hidden Agenda (1990),This Must Be The Place (2011),NaN,ALEC BALDWIN: THE SEXY YEARS,HORNINESS IS NOT A CRIME,VAMPIRES! FUCK!,ANIMALS,[Default to Nicolas Cage]
2,1/6/2021 10:36:44,Hidden Agenda (1990),Nomadland (2020),This Must Be The Place (2011),Raising Arizona (1987),Lone Star (1996),VAMPIRES! FUCK!,HORNINESS IS NOT A CRIME,ANIMALS,ALEC BALDWIN: THE SEXY YEARS,[Default to Nicolas Cage]
3,1/6/2021 12:00:31,Raising Arizona (1987),This Must Be The Place (2011),Lone Star (1996),Hidden Agenda (1990),Nomadland (2020),HORNINESS IS NOT A CRIME,[Default to Nicolas Cage],ALEC BALDWIN: THE SEXY YEARS,VAMPIRES! FUCK!,ANIMALS
4,1/7/2021 8:52:24,Lone Star (1996),This Must Be The Place (2011),Raising Arizona (1987),Nomadland (2020),Hidden Agenda (1990),[Default to Nicolas Cage],VAMPIRES! FUCK!,HORNINESS IS NOT A CRIME,ANIMALS,ALEC BALDWIN: THE SEXY YEARS


In [3]:
just_ballots = df.loc[:, df.columns.str.startswith('Rank')]
next_week = df.loc[:, df.columns.str.startswith('What')]

In [4]:
submissions = just_ballots.loc[just_ballots.notna().all(axis=1).idxmax(),:].values
submissions

array(['Nomadland (2020)', 'Raising Arizona (1987)',
       'This Must Be The Place (2011)', 'Hidden Agenda (1990)',
       'Lone Star (1996)'], dtype=object)

In [5]:
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}

In [6]:
balloted = just_ballots.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)

In [7]:
submission_dict

{'Nomadland (2020)': <Candidate('Nomadland (2020)')>,
 'Raising Arizona (1987)': <Candidate('Raising Arizona (1987)')>,
 'This Must Be The Place (2011)': <Candidate('This Must Be The Place (2011)')>,
 'Hidden Agenda (1990)': <Candidate('Hidden Agenda (1990)')>,
 'Lone Star (1996)': <Candidate('Lone Star (1996)')>}

In [8]:
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))

In [9]:
ballots

[<Ballot(Nomadland (2020), Raising Arizona (1987), This Must Be The Place (2011), Hidden Agenda (1990), Lone Star (1996))>,
 <Ballot(Lone Star (1996), Nomadland (2020), Hidden Agenda (1990), This Must Be The Place (2011))>,
 <Ballot(Hidden Agenda (1990), Nomadland (2020), This Must Be The Place (2011), Raising Arizona (1987), Lone Star (1996))>,
 <Ballot(Raising Arizona (1987), This Must Be The Place (2011), Lone Star (1996), Hidden Agenda (1990), Nomadland (2020))>,
 <Ballot(Lone Star (1996), This Must Be The Place (2011), Raising Arizona (1987), Nomadland (2020), Hidden Agenda (1990))>]

In [10]:
election_result = pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,)

In [11]:
print(election_result)

ROUND 1
Candidate                        Votes  Status
-----------------------------  -------  --------
Lone Star (1996)                     2  Hopeful
Nomadland (2020)                     1  Hopeful
Raising Arizona (1987)               1  Hopeful
Hidden Agenda (1990)                 1  Rejected
This Must Be The Place (2011)        0  Rejected

ROUND 2
Candidate                        Votes  Status
-----------------------------  -------  --------
Nomadland (2020)                     2  Hopeful
Lone Star (1996)                     2  Hopeful
Raising Arizona (1987)               1  Rejected
Hidden Agenda (1990)                 0  Rejected
This Must Be The Place (2011)        0  Rejected

FINAL RESULT
Candidate                        Votes  Status
-----------------------------  -------  --------
Lone Star (1996)                     3  Elected
Nomadland (2020)                     2  Rejected
Raising Arizona (1987)               0  Rejected
Hidden Agenda (1990)                 0  Rejected
T

In [12]:
submissions = next_week.loc[next_week.notna().all(axis=1).idxmax(),:].values
submission_dict = {submission: pyrankvote.Candidate(submission) for submission in submissions}
balloted = next_week.applymap(lambda x: submission_dict[x] if isinstance(x, str) else x)
ballots = []
for ballot in balloted.itertuples():
    ballots.append(pyrankvote.Ballot(ranked_candidates=[b for b in ballot if isinstance(b, pyrankvote.Candidate)]))
print(pyrankvote.instant_runoff_voting(submission_dict.values(), ballots,))

ROUND 1
Candidate                       Votes  Status
----------------------------  -------  --------
HORNINESS IS NOT A CRIME            1  Hopeful
[Default to Nicolas Cage]           1  Hopeful
VAMPIRES! FUCK!                     1  Hopeful
ALEC BALDWIN: THE SEXY YEARS        1  Hopeful
ANIMALS                             1  Rejected

ROUND 2
Candidate                       Votes  Status
----------------------------  -------  --------
[Default to Nicolas Cage]           2  Hopeful
HORNINESS IS NOT A CRIME            1  Hopeful
VAMPIRES! FUCK!                     1  Hopeful
ALEC BALDWIN: THE SEXY YEARS        1  Rejected
ANIMALS                             0  Rejected

ROUND 3
Candidate                       Votes  Status
----------------------------  -------  --------
HORNINESS IS NOT A CRIME            2  Hopeful
[Default to Nicolas Cage]           2  Hopeful
VAMPIRES! FUCK!                     1  Rejected
ALEC BALDWIN: THE SEXY YEARS        0  Rejected
ANIMALS                      